In [271]:
import cv2
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [272]:
img1 = cv2.imread('000000_10_left.png', cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('000000_10_right.png', cv2.IMREAD_GRAYSCALE)

In [273]:
def detect_and_match_features(img1, img2, num_matches=50):
    sift = cv2.SIFT_create()
    kp1, descriptors1 = sift.detectAndCompute(img1, None)
    kp2, descriptors2 = sift.detectAndCompute(img2, None)

    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
    matches = bf.match(descriptors1, descriptors2)
    matches = sorted(matches, key=lambda x: x.distance)[:num_matches]

    return kp1, kp2, matches

kp1, kp2, matches = detect_and_match_features(img1, img2)
img_matches = cv2.drawMatches(img1, kp1, img2, kp2, matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
cv2.imwrite('matches.png', img_matches)

True

In [274]:
def find_fundamental(kp1, kp2, matches):
    pts1 = np.int32([kp1[m.queryIdx].pt for m in matches])
    pts2 = np.int32([kp2[m.trainIdx].pt for m in matches])
    F, inliers = cv2.findFundamentalMat(pts1, pts2, cv2.FM_RANSAC)

    pts1 = pts1[inliers.ravel() == 1]
    pts2 = pts2[inliers.ravel() == 1]
    return F, pts1, pts2

F, pts1, pts2 = find_fundamental(kp1, kp2, matches)

In [275]:
def rectify(img1, pts1, img2, pts2):
  h1, w1 = img1.shape
  h2, w2 = img2.shape
  _, H1, H2 = cv2.stereoRectifyUncalibrated(np.float32(pts1), np.float32(pts2), F, imgSize=(h1, w1))
  print(H1)
  print(H2)

  img1r = cv2.warpPerspective(img1, H1, (w1, h1))
  img2r = cv2.warpPerspective(img2, H2, (w2, h2))
  cv2.imwrite('img1r.png', img1r)
  cv2.imwrite('img2r.png', img2r)
  return img1r, img2r

img1r, img2r = rectify(img1, pts1, img2, pts2)

[[-4.19250548e+11  3.46779934e+10  2.61484886e+12]
 [ 4.87898705e-02 -4.12834609e+11  3.30270493e+01]
 [ 3.89302751e-04  9.35775728e-16 -4.12834609e+11]]
[[ 1.00000000e+00 -4.40856708e-13  2.73331580e-10]
 [ 4.40856708e-13  1.00000000e+00 -8.24229573e-11]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [276]:
def stereo_block_matching(img1r, img2r, window_size=3, max_disparity=64):
    height, width = img1r.shape
    disparity_map = np.zeros((height, width), np.float32)
    ssd_scores = np.full((height, width, max_disparity), np.inf)
    kernel = np.ones((window_size, window_size), dtype=np.float32)

    left_sq = img1r.astype(np.float32) ** 2
    left_sq_sum = cv2.filter2D(left_sq, -1, kernel)

    for d in range(max_disparity):
        shifted_right = np.roll(img2r, -d, axis=1)
        shifted_right[:, -d:] = 0

        right_sq = shifted_right.astype(np.float32) ** 2
        right_sq_sum = cv2.filter2D(right_sq, -1, kernel)
        cross_term = cv2.filter2D(img1r.astype(np.float32) * shifted_right, -1, kernel)

        ssd = left_sq_sum - 2*cross_term + right_sq_sum
        ssd_scores[:, :, d] = ssd

    disparity_map = np.argmin(ssd_scores, axis=2)
    return disparity_map

disparities = stereo_block_matching(img1r, img2r, window_size=5, max_disparity=64)
print(disparities)

[[ 1  1  1 ...  1  1  1]
 [ 1  1  1 ...  1  1  1]
 [ 1  1  1 ...  1  1  1]
 ...
 [27 28 18 ...  0  0  0]
 [ 8  7  6 ...  0  0  0]
 [ 7  7  5 ...  0  0  0]]


In [277]:
disparity_normalized = cv2.normalize(disparities, None, 0, 255, cv2.NORM_MINMAX)
disparity_normalized = np.uint8(disparity_normalized)

print(disparity_normalized)
cv2.imwrite('output.png', disparity_normalized)

[[  4   4   4 ...   4   4   4]
 [  4   4   4 ...   4   4   4]
 [  4   4   4 ...   4   4   4]
 ...
 [109 113  73 ...   0   0   0]
 [ 32  28  24 ...   0   0   0]
 [ 28  28  20 ...   0   0   0]]


True